In [ ]:
#!/usr/bin/env python3
# test_integrity.py

import os
import pandas as pd
from pymongo import MongoClient
from dotenv import load_dotenv

def main():
    # 1. Charge le .env et écrase les vars système si besoin
    load_dotenv(override=True)
    
    # 2. Récupère les variables d'environnement
    MONGO_URI       = os.getenv("MONGO_URI")
    DB_NAME         = os.getenv("MONGO_DB")
    COLL_NAME       = os.getenv("MONGO_COLLECTION")
    CSV_PATH        = os.getenv("CSV_PATH", "data/healthcare_dataset.csv")
    
    # 3. Affiche pour vérification
    print("🔌 Mongo URI     :", MONGO_URI)
    print("📁 Database     :", DB_NAME)
    print("📂 Collection   :", COLL_NAME)
    print("📄 CSV Path     :", CSV_PATH)
    print()
    
    # 4. Connexion MongoDB
    client     = MongoClient(MONGO_URI)
    db         = client[DB_NAME]
    collection = db[COLL_NAME]
    
    # 5. Chargement du CSV
    df_csv     = pd.read_csv(CSV_PATH)
    
    # 6. Chargement des documents MongoDB
    docs       = list(collection.find({}, {"_id": 0}))
    df_mongo   = pd.DataFrame(docs)
    
    # 7. Vérifications
    print("=== VÉRIFICATIONS D'INTÉGRITÉ ===")
    
    # 7.1 Nombre de lignes
    n_csv   = len(df_csv)
    n_mongo = len(df_mongo)
    print(f"• Lignes CSV    : {n_csv}")
    print(f"• Documents DB  : {n_mongo}")
    assert n_csv == n_mongo, f"❌ Incohérence count : CSV={n_csv}, Mongo={n_mongo}"
    
    # 7.2 Colonnes
    cols_csv   = set(df_csv.columns)
    cols_mongo = set(df_mongo.columns)
    print(f"• Colonnes CSV  : {cols_csv}")
    print(f"• Colonnes DB   : {cols_mongo}")
    assert cols_csv == cols_mongo, f"❌ Colonnes diff : {cols_csv ^ cols_mongo}"
    
    # 7.3 Types de données
    dtypes_csv   = df_csv.dtypes.to_dict()
    dtypes_mongo = df_mongo.dtypes.to_dict()
    print("• Types CSV     :", dtypes_csv)
    print("• Types DB      :", dtypes_mongo)
    # (optionnel) vérifier que les types pandas sont compatibles
    
    # 7.4 Valeurs manquantes
    na_csv   = df_csv.isnull().sum().to_dict()
    na_mongo = df_mongo.isnull().sum().to_dict()
    print("• Manquants CSV :", na_csv)
    print("• Manquants DB  :", na_mongo)
    assert na_csv == na_mongo, "❌ Différence dans les valeurs manquantes"
    
    # 7.5 Doublons
    dup_csv   = int(df_csv.duplicated(keep=False).sum())
    dup_mongo = int(df_mongo.duplicated(keep=False).sum())
    print(f"• Doublons CSV  : {dup_csv}")
    print(f"• Doublons DB   : {dup_mongo}")
    assert dup_csv == dup_mongo, f"❌ Doublons diff : CSV={dup_csv}, DB={dup_mongo}"
    
    print("\n✅ Tous les tests d’intégrité sont passés avec succès.")

if __name__ == "__main__":
    main()


AssertionError: Incohérence : CSV = 55500, MongoDB = 444000